In [1]:
from app import create_app
app = create_app()
app_context =  app.app_context()
app_context.push()
# app_context.pop()

In [2]:
from app.models1 import *
from app.models import *
import random
from datetime import datetime


In [3]:
c = Channel.query.filter_by(id=1).first()
print(c)

<Channel 1>


各个机场代码如下：其中山东临沂机场没有代码，而南宁吴圩国际机场和上海浦东国际机场有两个，在存数据库时用其中一个
```
ZYTL	大连周水子国际机场
ZGGG	广州白云国际机场
ZSHC	杭州萧山国际机场
WHJC	武汉机场
ZGSD	珠港机场
JYCS	揭阳潮汕机场
nng	南宁吴圩国际机场
NNG	南宁吴圩国际机场
#	山东临沂机场
ZSPD	上海浦东国际机场
ZSPDbs	上海浦东国际机场
ZGSZ	深圳机场
ZBTJ	天津滨海国际机场
YW	浙江省义乌机场
ZHCC	郑州新郑国际机场
ZUCK	重庆机场集团公司飞行区管理部
SCX	长沙黄花国际机场
CZX	常州国际机场有限公司
XZ	徐州机场
ZBAD	北京大兴国际机场
ZSNT	南通兴东国际机场
```


In [4]:
code1 = ['ZYTL','ZGGG','ZSHC','WHJC','ZGSD','JYCS','NNG','ZSPD',
        'ZGSZ','ZBTJ','YW','ZHCC','ZUCK','SCX','CZX','XZ','ZBAD','ZSNT']
name1 = ['大连周水子国际机场','广州白云国际机场','杭州萧山国际机场','武汉机场','珠港机场','揭阳潮汕机场',
        '南宁吴圩国际机场','上海浦东国际机场','深圳机场',
        '天津滨海国际机场','浙江省义乌机场','郑州新郑国际机场','重庆机场集团公司飞行区管理部',
        '长沙黄花国际机场','常州国际机场有限公司','徐州机场','北京大兴国际机场','南通兴东国际机场']
len(name1)

18

# 加入各个机场（渠道）信息
for i in range(18):
    c = Channel.query.filter_by(code=code1[i]).first()
    if not c :
        c = Channel(code1[i],name1[i])
        db.session.add(c)
db.session.commit()

# 创建用户信息：
for i in range(18):
    u = User.query.filter_by(username=name1[i]).first()
    if not u :
        u = User()
        u.username = name1[i]
        u.set_password(code1[i])
        
        c = Channel.query.filter_by(code=code1[i]).first()
        u.channel =c 
        db.session.add(u)
db.session.commit()

In [5]:
User.query.all()

[<User 大连周水子国际机场>,
 <User 广州白云国际机场>,
 <User 杭州萧山国际机场>,
 <User 武汉机场>,
 <User 珠港机场>,
 <User 揭阳潮汕机场>,
 <User 南宁吴圩国际机场>,
 <User 上海浦东国际机场>,
 <User 深圳机场>,
 <User 天津滨海国际机场>,
 <User 浙江省义乌机场>,
 <User 郑州新郑国际机场>,
 <User 重庆机场集团公司飞行区管理部>,
 <User 长沙黄花国际机场>,
 <User 常州国际机场有限公司>,
 <User 徐州机场>,
 <User 北京大兴国际机场>,
 <User 南通兴东国际机场>]

In [6]:
# 由于测序数据的code并不统一，有些机场有两个编码：
code = ['ZYTL','ZGGG','ZSHC','WHJC','ZGSD','JYCS','nng','NNG','ZSPD','ZSPDbs',
        'ZGSZ','ZBTJ','YW','ZHCC','ZUCK','SCX','CZX','XZ','ZBAD','ZSNT']
name = ['大连周水子国际机场','广州白云国际机场','杭州萧山国际机场','武汉机场','珠港机场','揭阳潮汕机场',
        '南宁吴圩国际机场','南宁吴圩国际机场','上海浦东国际机场','上海浦东国际机场','深圳机场',
        '天津滨海国际机场','浙江省义乌机场','郑州新郑国际机场','重庆机场集团公司飞行区管理部',
        '长沙黄花国际机场','常州国际机场有限公司','徐州机场','北京大兴国际机场','南通兴东国际机场']
len(name)

20

In [7]:
airport_dict = dict(zip(code,name))
print(airport_dict)

{'ZYTL': '大连周水子国际机场', 'ZGGG': '广州白云国际机场', 'ZSHC': '杭州萧山国际机场', 'WHJC': '武汉机场', 'ZGSD': '珠港机场', 'JYCS': '揭阳潮汕机场', 'nng': '南宁吴圩国际机场', 'NNG': '南宁吴圩国际机场', 'ZSPD': '上海浦东国际机场', 'ZSPDbs': '上海浦东国际机场', 'ZGSZ': '深圳机场', 'ZBTJ': '天津滨海国际机场', 'YW': '浙江省义乌机场', 'ZHCC': '郑州新郑国际机场', 'ZUCK': '重庆机场集团公司飞行区管理部', 'SCX': '长沙黄花国际机场', 'CZX': '常州国际机场有限公司', 'XZ': '徐州机场', 'ZBAD': '北京大兴国际机场', 'ZSNT': '南通兴东国际机场'}


In [11]:
# old_date和old_code的目的是为了判断是否是同一个订单同一个机场
old_code = ''
old_date = ''
old_order  = None
i = 0
with open('airport.csv') as fh,open('cannot_find_code.tsv','w') as out:
    header  = fh.readline()
    for line in fh:
        line = line.strip()
        if line:
            lst = line.split(',')
#             print(lst)
            date = lst[1]
#             sample_date = datetime.strptime(date,'%Y%m%d')
            sample_date = datetime.now()
            code = lst[2]
            gene = lst[3]
            path = lst[5]
            # 
#             data = SeqData(path,gene,'F',sample_date,'')
            data = SeqData(path,gene,'F',sample_date,'')

#             break

            channel = Channel.query.filter_by(code=code).first()
            if channel:
                if old_date == date: #相同则说明是同一天下单测序的：
                    if old_code == code : #相同则说明是同一个订单同一个机场（渠道）
                        order = old_order
                        order.seqData.append(data)
                    else:
                        randnum = str(random.randint(1000, 9999))
                        order_num = date+'_'+randnum
                        order = OrderSeq(order_num,sample_date)
                        db.session.add(order)
                else: # 不同一天，就建立新的订单：  
                    randnum = str(random.randint(1000, 9999))
                    order_num = date+'_'+randnum
                    order = OrderSeq(order_num,sample_date)

                u = User.query.filter_by(username = channel.name).first()
                if order not in u.orders_seq:
                    u.orders_seq.append(order)
                db.session.add(data)
            else:
                #找不到对应的编码的机场就记录下来：
                out.write(line+'\n')

            old_date = date
            old_code = code
            old_order = order
            

db.session.commit()            

In [19]:
date = '20200427'

In [20]:
datetime.now()

datetime.datetime(2021, 5, 28, 15, 19, 55, 431634)

In [21]:
datetime.strptime(date,'%Y%m%d')

datetime.datetime(2020, 4, 27, 0, 0)

In [18]:
us = User.query.all()
for  i in us:
# u
    print(i.orders_seq.all())

[]
[<OrderSeq 7>, <OrderSeq 14>, <OrderSeq 19>, <OrderSeq 74>, <OrderSeq 76>, <OrderSeq 78>, <OrderSeq 80>, <OrderSeq 82>, <OrderSeq 84>, <OrderSeq 86>, <OrderSeq 88>, <OrderSeq 90>, <OrderSeq 92>, <OrderSeq 94>, <OrderSeq 96>, <OrderSeq 104>, <OrderSeq 176>, <OrderSeq 178>, <OrderSeq 180>, <OrderSeq 182>]
[<OrderSeq 1>, <OrderSeq 3>, <OrderSeq 5>, <OrderSeq 13>, <OrderSeq 15>, <OrderSeq 16>, <OrderSeq 21>, <OrderSeq 23>, <OrderSeq 26>, <OrderSeq 39>, <OrderSeq 41>, <OrderSeq 43>, <OrderSeq 45>, <OrderSeq 47>, <OrderSeq 49>, <OrderSeq 51>, <OrderSeq 53>, <OrderSeq 55>, <OrderSeq 57>, <OrderSeq 59>, <OrderSeq 61>, <OrderSeq 63>, <OrderSeq 65>, <OrderSeq 67>, <OrderSeq 68>, <OrderSeq 70>, <OrderSeq 72>, <OrderSeq 75>, <OrderSeq 77>, <OrderSeq 79>, <OrderSeq 81>, <OrderSeq 83>, <OrderSeq 85>, <OrderSeq 87>, <OrderSeq 89>, <OrderSeq 91>, <OrderSeq 93>, <OrderSeq 95>, <OrderSeq 97>, <OrderSeq 98>, <OrderSeq 99>, <OrderSeq 101>, <OrderSeq 103>, <OrderSeq 105>, <OrderSeq 106>, <OrderSeq 107>,